In [1]:
import huggingface_hub
import os

from bitmind.synthetic_image_generation.synthetic_image_generator import SyntheticImageGenerator
from bitmind.image_dataset import ImageDataset
from bitmind.utils.data import sample_dataset_index_name
from bitmind.constants import DATASET_META, HUGGINGFACE_CACHE_DIR
import time

2024-08-22 18:18:02.910670: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-22 18:18:02.930432: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 18:18:02.948804: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 18:18:02.954313: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 18:18:02.968694: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print("Loading real datasets")
real_image_datasets = [
    ImageDataset(ds['path'], 'train', ds.get('name', None), ds['create_splits'], 'REUSE_CACHE_IF_EXISTS')
    for ds in DATASET_META['real'][:1]
]


Loading real datasets
Downloaded bitmind/open-images-v7 dataset to /workspace/.cache/huggingface


In [3]:
start_time = time.time()
synthetic_image_generator = SyntheticImageGenerator(
    prompt_type='annotation', 
    use_random_diffuser=False, 
    diffuser_name='black-forest-labs/FLUX.1-dev')

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Initialization of SyntheticImageGenerator took {elapsed_time:.2f} seconds.")

Loading meta-llama/Meta-Llama-3.1-8B-Instruct
Done loading moderation pipeline
Initialization of SyntheticImageGenerator took 28.44 seconds.


In [ ]:
import matplotlib.pyplot as plt

times = []

for _ in range(5):
    real_dataset_index, source_dataset = sample_dataset_index_name(real_image_datasets)
    real_dataset = real_image_datasets[real_dataset_index]
    images_to_caption, image_indexes = real_dataset.sample(k=1)  # [{'image': PIL Image ,'id': int}, ...]
    
    # generate captions for the real images, then synthetic images from these captions
    sample = synthetic_image_generator.generate(
        k=1, real_images=images_to_caption)[0]  # {'prompt': str, 'image': PIL Image ,'id': int}

    print(sample['prompt'])
    plt.imshow(sample['image'])
    plt.show()

    times.append(sample['gen_time'])

In [ ]:
import numpy as np
np.mean(times), np.std(times)

In [ ]:
sample['image'].size

In [ ]:
from bitmind.constants import VALIDATOR_MODEL_META

VALIDATOR_MODEL_META


In [ ]:
sample

In [ ]:
sample

In [ ]:
import transformers
print(transformers.__version__)